__собираем предложения о земельных участках__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# каталог с файлами объявлений
avito_raw_data_path = 'data/raw/land/avito' 
# cian_raw_data_path = 'data/raw/cian'
result_file_path = 'data/data_land.pkl'

In [5]:
# ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
# ts

## собираем данные авито.ру

In [6]:
import re
from os import listdir

raw_data_files = sorted([
        avito_raw_data_path+'/'+f 
        for f in listdir(avito_raw_data_path) 
        if re.match(r'.+\.xlsx$',f)
    ])
raw_data_files

['data/raw/land/avito/avito_2022-09-08_16-05_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-12_13-51_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-13_13-56_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-15_10-31_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-16_15-10_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-19_12-39_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-22_09-57_raw.xlsx',
 'data/raw/land/avito/avito_2022-09-26_11-22_raw.xlsx',
 'data/raw/land/avito/avito_2022-10-08_20-38_raw.xlsx',
 'data/raw/land/avito/avito_2022-10-14_15-54_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-01_13-58_raw.xlsx',
 'data/raw/land/avito/avito_2022-11-04_17-12_raw.xlsx']

In [7]:
# for f in raw_data_files:
#     df = pd.read_excel(f)
#     df['place']='sevastopol'
#     df.to_excel(f,index=False)
#     print(f)

In [8]:
data_avito = pd.concat([
        pd
        .read_excel(f)
        #.dropna()
        .drop_duplicates(['avito_id','description'])
        .rename(columns={'page':'avito_page'})
        .reset_index(drop=True)  
        for f in raw_data_files
    ]).reset_index(drop=True)

print(len(data_avito))

13062


In [9]:
# data_avito

In [10]:
from lib.avito import AvitoDataCleanerRealtyLand

data_avito = AvitoDataCleanerRealtyLand().transform( data_avito )

print(len(data_avito))

13062


In [11]:
data_avito.sample(3)

,avito_id,title,price,obj_name,adr,description,avito_page,ts,place,area,is_IJS,priceM,priceMU,area_size_category
6185,2500644601,"Участок 4,6 сот. (ИЖС)",9800000,,"Крайняя ул., 13","Продажа участка Ижс в Гагаринском районе. Район ЖК Тульский квартал, ЖК Руднево, ТЦ Муссон. Коммуникации, удобный подьезд. Отличное расположение участка, близость к развитой инфраструктуре (торговые центры, школа, детский сад), вид на море (Стрелецкая, Пес",14,2022-09-19 12:39:53.571,sevastopol,4.60,True,9.80,2.13,4-8
3931,1329752453,Участок 6 сот. (ИЖС),2700000,,Качинское ш.,"Севастополь, микрорайон Любимовка. \n\nПродам участок 6 соток, ровный, без перепадов высоты. \n\nЧастная собственность, все российские документы. \n\nРядом пляж Любимовка (500-700 м) в пешей доступности (5-10 минут). Пляж Любимовка — самый большой песчаный морск",13,2022-09-15 10:31:26.039,sevastopol,6.00,True,2.70,0.45,4-8
10690,2368313504,Участок 10 сот. (ИЖС),4500000,,ул. Богданова,"Предлагаем Вашему вниманию участок 10 соток по адресу: г. Севастополь, Нахимовский район, ул. Богданова.\nУчасток ровный. В 40 м от основной дороги. \n\nКоммуникации по границе: \n- Электроэнергия — 15квт.\n- Вода городская.\n\nКатегория земель- земли населённых",16,2022-10-14 15:54:34.254,sevastopol,10.00,True,4.50,0.45,8-20


In [12]:
assert len(data_avito)>0

In [13]:
data_avito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13062 entries, 0 to 13061
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   avito_id            13062 non-null  int64         
 1   title               13062 non-null  object        
 2   price               13062 non-null  int64         
 3   obj_name            13062 non-null  object        
 4   adr                 13062 non-null  object        
 5   description         13062 non-null  object        
 6   avito_page          13062 non-null  int64         
 7   ts                  13062 non-null  datetime64[ns]
 8   place               13062 non-null  object        
 9   area                13062 non-null  float64       
 10  is_IJS              13062 non-null  bool          
 11  priceM              13062 non-null  float64       
 12  priceMU             13062 non-null  float64       
 13  area_size_category  13057 non-null  category  

---

In [14]:
data_avito['url'] = 'https://avito.ru/' + data_avito['avito_id'].astype(int).astype(str)
data_avito = data_avito.drop(columns=['avito_id'])
# data_cian = data_cian.rename(columns={'cian_url':'url'})

In [15]:
data = data_avito.reset_index(drop=True) 
print(len(data))
assert len(data)>0

13062


## обновляем таблицу адресов

In [16]:
# data_ = data.sample(15).reset_index(drop=True)
# data_[['adr']]

In [17]:
places = { 
# поле 'place' и название города
'bahchisaray': 'Бахчисарай,',
      'sudak': 'Судак,',
  'sevastopol': 'Севастополь,',
}

def norm_place(d,places=places):
    if not d['place'] in places: return d['adr_']
    p = places[ d['place'] ]
    return d['adr_'] if d['adr_'].find(p)>-1 else p+d['adr_']

In [18]:
from lib.locator import AddressTransformer

atr = AddressTransformer()
data['adr_'] = data['adr'].progress_apply(atr.transform)
data['adr_'] = data[['adr_','place']].progress_apply(norm_place,axis=1)
data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

  0%|          | 0/13062 [00:00<?, ?it/s]

  0%|          | 0/13062 [00:00<?, ?it/s]

In [19]:
data[['adr_orig','adr']].sample(10)

,adr_orig,adr
5827,"Нахимовский муниципальный округ, посёлок Любимовка, квартал Совхоз имени Софьи Перовской","Севастополь,Нахимовский район , Любимовка, квартал Совхоз имени Софьи Перовской"
2460,"ул. Разведчика Копылова, 8","Севастополь,ул. Разведчика Копылова, 8"
5881,Каспийская ул.,"Севастополь,Каспийская ул."
3966,"садоводческое товарищество Бельбек, 1","Севастополь,садоводческое товарищество Бельбек, 1"
9637,"с. Широкое, ул. Клюшкина, 61","Севастополь,с. Широкое, ул. Клюшкина, 61"
32,ТСН СТ Дружба,"Севастополь,ТСН СТ Дружба"
12025,"с. Андреевка, Приусадебная ул., 9","Севастополь,с. Андреевка, Приусадебная ул., 9"
491,2-й Голландский пер.,"Севастополь,2-й Голландский пер."
2124,"с. Осипенко, ул. Землянского, 11","Севастополь,с. Осипенко, ул. Землянского, 11"
9847,район Северная Сторона,"Севастополь,район Северная Сторона"


In [20]:
# from lib.locator import GeocoderSimpleOSM
from lib.locator import GeocoderYandex
from yandex_keys import keys

atr = AddressTransformer()
locator = GeocoderYandex(keys[0])
# locator = GeocoderSimpleOSM()

loc_file_path = 'data/location.pkl' # таблица адресов
loc = pd.read_pickle(loc_file_path)
# loc = pd.DataFrame([],columns=['adr','latitude','longitude',])
logging.info(f'LocationUpdater: {len(loc)} addresses in index')

# собираем все адреса в один список
loc = loc.merge(
        data[['adr']].drop_duplicates(),
        how='outer',
        on='adr',
    ).drop_duplicates().reset_index(drop=True)

logging.info(f'LocationUpdater: {len(loc)} addresses total')

# собираем все адреса с геопозицией
loc_def = loc[~loc['latitude'].isna()].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_def)} addresses defined')

# собираем все адреса без геопозиции
loc_undef = loc[loc['latitude'].isna()][['adr']].reset_index(drop=True)
logging.info(f'LocationUpdater: {len(loc_undef)} addresses undefined')

if len(loc_undef)>0:
    # определяем координаты по строке адреса
    loc_undef = loc_undef.join( loc_undef['adr'].progress_apply(locator.transform).apply(pd.Series) ).dropna()     
    logging.info(f'LocationUpdater: {len(loc_undef)} new addresses found')
    loc = pd.concat([loc_def,loc_undef]).reset_index(drop=True) 
    assert len(loc)>0
    loc.to_pickle(loc_file_path)

[INFO    ] 2022-11-04 17:16:40 | LocationUpdater: 6999 addresses in index
[INFO    ] 2022-11-04 17:16:40 | LocationUpdater: 7016 addresses total
[INFO    ] 2022-11-04 17:16:40 | LocationUpdater: 6999 addresses defined
[INFO    ] 2022-11-04 17:16:40 | LocationUpdater: 17 addresses undefined


  0%|          | 0/17 [00:00<?, ?it/s]

[INFO    ] 2022-11-04 17:16:44 | LocationUpdater: 17 new addresses found


In [21]:
print( len(loc) )
loc.sample(3)

7016


,latitude,longitude,adr
2664,44.734977,33.546291,"Севастополь, район Нахимовский, село Орловка, шоссе Качинское, 33Б/1"
2037,44.718513,33.550612,"Севастополь, село Орловка, Качинское шоссе , 45А"
5435,44.899217,33.616189,"Севастополь,с. Береговое, Приморская ул., 9"


## дополняем данные геометкой

In [22]:
# data = data.rename(columns={'adr':'adr_orig','adr_':'adr'})

data = data.merge(loc,on=['adr'],how='left')

print('всего записей:', len( data) )
print('записей без геометки:', len( data[ data['latitude'].isna() ] ) )

assert len(data)>0
data.to_pickle(result_file_path)

data.info()

всего записей: 13062
записей без геометки: 0
<class 'pandas.core.frame.DataFrame'>
Int64Index: 13062 entries, 0 to 13061
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   title               13062 non-null  object        
 1   price               13062 non-null  int64         
 2   obj_name            13062 non-null  object        
 3   adr_orig            13062 non-null  object        
 4   description         13062 non-null  object        
 5   avito_page          13062 non-null  int64         
 6   ts                  13062 non-null  datetime64[ns]
 7   place               13062 non-null  object        
 8   area                13062 non-null  float64       
 9   is_IJS              13062 non-null  bool          
 10  priceM              13062 non-null  float64       
 11  priceMU             13062 non-null  float64       
 12  area_size_category  13057 non-null  category      
 13  u